# <font color = 'blue'> __5 State Time Homogeneous Markov Chain - Code__

We consider a time homogeneous Markov Chain with the transition matrix $P(n) = P ,\text{      } \forall n$. We take the finite time $N = 5$. (Helfmann et al. 2020)
\begin{equation}
P = 
\begin{pmatrix}
0.7 & 0.2 & 0.0 & 0.1 & 0.0 \\
0.1 & 0 & 0.8 & 0 & 0.1 \\
0 & 0.4 & 0.6 & 0 & 0 \\
0.5 & 0 & 0 & 0 & 0.5 \\
0 & 0.2 & 0 & 0.1 & 0.7 
\end{pmatrix}
\end{equation}

## <font color = 'blue'> Directory
[1.1 Computation of Committors](#1.1)
- [1.1.1 Forward Committors](#1.1.1)
- [1.1.2 Backward Committors for the Stationary Distribution](#1.1.2)
- [1.1.3 Backward Committors for a General Initial Distribution](#1.1.3)

In [17]:
import numpy as np
from tpt_gc import reverse_process, forward_committors, backward_committors

## <a id='1.1'> <font color = 'blue'> 1.1 Computation of Committors </a>

In [25]:
P = np.array([[0.7, 0.2, 0.0, 0.1, 0.0], # Transition matrix of Markov Chain
             [0.1, 0.0, 0.8, 0, 0.1],
             [0.0, 0.4, 0.6, 0.0, 0.0],
             [0.5, 0.0, 0.0, 0.0, 0.5],
             [0.0, 0.2, 0.0, 0.1, 0.7]])
size = len(P)

pi_0 = [0.5, 0.5, 0, 0, 0] # Initial distribution

N = 5

A = [0]
B = [4]


## <a id='1.1.1'> <font color = 'blue'> 1.1.1 Forward Committors </a>

In [5]:
final_distribution = np.array([0, 0, 0, 0, 1])

# Initialise the committors from time T = 0 to T = 4
forward_committors = np.zeros((5,5))
P = np.array([[0.7, 0.2, 0.0, 0.1, 0.0], 
             [0.1, 0.0, 0.8, 0, 0.1],
             [0.0, 0.4, 0.6, 0.0, 0.0],
             [0.5, 0.0, 0.0, 0.0, 0.5],
             [0.0, 0.2, 0.0, 0.1, 0.7]])

for i in range(4, -1, -1):
    if i == 4:
        committor_n = final_distribution
    committor_n =  P @ committor_n
    committor_n[0] = 0
    committor_n[4] = 1
    forward_committors[i] = committor_n

print(forward_committors)

[[0.      0.17296 0.1152  0.5     1.     ]
 [0.      0.1512  0.0912  0.5     1.     ]
 [0.      0.132   0.064   0.5     1.     ]
 [0.      0.1     0.04    0.5     1.     ]
 [0.      0.1     0.      0.5     1.     ]]


In [13]:
from tpt_gc import forward_committors

q_plus = forward_committors(P, N, A, B)

### <a id='1.1.2'> <font color = 'blue'> 1.1.2 Backward Committors for the Invariant Distribution </a>

In [277]:
# Computation of the invariant distribution of P 
eigenvalues, eigenvectors = np.linalg.eig(P.T)
invariant = eigenvectors[:, eigenvalues == 1]
invariant = invariant / invariant.sum(axis=0)

invariant = invariant.reshape(5)
invariant

array([0.12195122, 0.24390244, 0.48780488, 0.02439024, 0.12195122])

In [319]:
P_reverse = np.zeros((5,5))

for i in range(5):
    for j in range(5):
        P_reverse[i,j] = P[j, i] * (invariant[j] / invariant[i])

In [325]:
# Computing the backward committors:
initial_distribution = np.array([1, 0, 0, 0, 0])

# Initialise the committors from time T = 1 to T = 5
backward_committors = np.zeros((5,5))

for i in range(5):
    if i == 0:
        backward_committor_n = initial_distribution
    backward_committor_n = P_reverse @ backward_committor_n 
    backward_committor_n[0] = 1
    backward_committor_n[4] = 0
    backward_committors[i] = backward_committor_n

print(backward_committors)

[[1.      0.1     0.      0.5     0.     ]
 [1.      0.1     0.04    0.5     0.     ]
 [1.      0.132   0.064   0.5     0.     ]
 [1.      0.1512  0.0912  0.5     0.     ]
 [1.      0.17296 0.1152  0.5     0.     ]]


In [ ]:
q_minus

### <a id='1.1.3'> <font color = 'blue'> 1.1.3 Backward Committors for the General Case </a>
In this section, an algorithm to determine which entries in the distribution need to be filled in to ensure well-defined time-reversal transition matrices was coded.

In [7]:
# Algorithm for determining which entries should be filled in, based on the intention of filling in the least number of rows in the distribution.
def least_nonzero_rows(matrix):
    zero_cols = np.array([True for i in range(len(matrix.T))]) 
    entries_to_fill = []

    while zero_cols.any():
    #    """
    #    max_non_zeros = 0
    #    max_non_zeros_idx = None
    #    for key, val in matrix_columns.items():
    #        non_zeros = np.count_nonzero(val != 0)
    #       if non_zeros > max_non_zeros:
    #            max_non_zero = non_zeros 
    #            max_non_zero_idx = key
    #    """
        non_zeros = np.sum(matrix[:, zero_cols] != 0, axis=1)
        entries_to_fill.append(np.argmax(non_zeros)) # Find the row with the most non-zero columns 
        # Index the row with the most non-zero entries
        zero_cols = np.where((matrix[np.argmax(non_zeros)] == 0) & (zero_cols == True), zero_cols, False) 
        
    return entries_to_fill
    
least_nonzero_rows(P)

[0, 1]

In [ ]:
P_reverse, distributions = reverse_process(P, pi_0, N)
    
q_minus = backward_committors(P_reverse, N, A, B)

## <a id='1.2'> <font color = 'blue'> 1.2 Visualisation of Statistics </a>

In [27]:
import matplotlib.pyplot as plt

reactive_distributions = np.zeros((N+1, size))

for i in range(N+1):
    reactive_distributions[i] = q_plus[i] * distributions[i] * q_minus[i]

In [32]:
for T in range(N):
    #plt.imshow(reactive_distributions[T])
    #plt.title(f'Probability of state and reactive at $T = {T}$')
    #plt.colorbar()
   # plt.show()
    print(reactive_distributions[T])

[0. 0. 0. 0. 0.]
[0.      0.01512 0.      0.025   0.     ]
[0.      0.01056 0.00512 0.02    0.     ]
[0.      0.0095  0.00448 0.01575 0.     ]
[0.      0.00984 0.      0.0134  0.     ]
